# Validation - Belchatow case

Shin et al (2020)

#### Packages

In [1]:
import json
from math import pi, ceil, sin, floor
from iapws.iapws97 import _Region4, _Region2, _Region1
from iapws._iapws import _ThCond, _Viscosity
from constructors import *
from scipy.optimize import root
import numpy as np
import os.path

#### Constants

In [2]:
g = 9.81 # m/s2 - gravitational acceleration
pi = pi # pi

#### Heat Exchanger Information

In [3]:
Do = 0.024 # m external diameter
Di = Do - 2*0.001 # m external diameter
n_tubes = 3220
arrange = "triangle"
#arrange = "square"

L = 9.94 # m tube length
W = 1.8 * 1 # m bundle width
ep = 45e-6 # m - roughness
kwall = 16.0 # W / ( m * K )
pitch = 0.0333 # m
Rows = 15
row_pitch = pitch
tubes_per_width = n_tubes/Rows
ResF= 5.82e-5 # K/W

N_wetters = []
row_index = np.linspace(1,Rows,num=Rows)
for i in row_index:
    if arrange == "triangle":
        N_wetters.append(ceil(i/2)-1)
    else:
        N_wetters = row_index-1
N_wetters = np.array(N_wetters)        

print("Number of tubes per row are ", tubes_per_width)
print("Number of rows is {number:.{digits}f}".format(number=Rows,digits=0))
print("Number of tubes is {number:.{digits}f}".format(number=Rows*tubes_per_width,digits=0))


Number of tubes per row are  214.66666666666666
Number of rows is 15
Number of tubes is 3220


In [4]:
# Fouling
kF = 2.941 # W / ( m * K ) (Calcium carbonate)
Df = Di * np.exp(- ResF * 2 * pi * kF)
lf = 1e6*(Di-Df)/2  # um fouling thickness

print("Wall internal diameter is {number:.{digits}f}".format(number=Di,digits=4))
print("Fouling internal diameter is {number:.{digits}f}".format(number=Df,digits=6))
print("Fouling thickness is {number:.{digits}f} um".format(number=1e6*(Di-Df)/2,digits=0))

Wall internal diameter is 0.0220
Fouling internal diameter is 0.021976
Fouling thickness is 12 um


#### Operational Conditions

In [5]:
Tin = 22.4 + 273.15 # K
Pin = 200000 # Pa
Tvap_expected = 37.96 + 273.15
Tout_mean_expected = 30.2 + 273.15
Tout = Tout_mean_expected * np.ones(Rows) # K
Pout = 156000 * np.ones(Rows) # Pa
m = 2448 # kg/s cooling Water
Pvap = 6510 # Pa
Tvap = _Region4(Pvap*1e-6, 0.)['T']

# Results
print("The exhaust steam pressure is equal to {0} Pa."\
      .format(Pvap))
print("The exhaust steam temperature is equal to {number:.{digits}f}°C."\
      .format(number=Tvap-273.15, digits=2))

The exhaust steam pressure is equal to 6510 Pa.
The exhaust steam temperature is equal to 37.66°C.


#### Water Properties

In [6]:
# Calculating properties
Tm = 0.5 * (Tin + np.mean(Tout)) # K - mean cooling water temperature
Pm = 0.5 * (Pin + np.mean(Pout)) # Pa - mean cooling water pressure
water_properties = _Region1(Tm,1e-6*Pm)
water_density = 1/water_properties['v']
water_entalphy = 1e3*water_properties['h']
water_heat_capacity = 1e3*water_properties['cp']
water_conductivity = _ThCond(water_density, Tm)
water_viscosity = _Viscosity(water_density, Tm)
water_prandtl = water_heat_capacity * water_viscosity / water_conductivity

# Results
print("Estimated mean cooling water temperature is {number:.{digits}f} K".format(number=Tm,digits=2))
print("The water density is {number:.{digits}f} kg/m3.".format(number=water_density, digits=2))
print("The water entalphy is {number:.{digits}f} J/kg.".format(number=water_entalphy, digits=2))
print("The water heat capacity is {number:.{digits}f} J/(kg*K).".format(number=water_heat_capacity, digits=2))
print("The water conductivity is {number:.{digits}f} W/(m*K).".format(number=water_conductivity, digits=3))
print("The water viscosity is {number:.{digits}f} (Pa*s).".format(number=water_viscosity, digits=5))
print("The water Prandtl is {number:.{digits}f}.".format(number=water_prandtl, digits=5))

Estimated mean cooling water temperature is 299.45 K
The water density is 996.74 kg/m3.
The water entalphy is 110436.10 J/kg.
The water heat capacity is 4181.10 J/(kg*K).
The water conductivity is 0.609 W/(m*K).
The water viscosity is 0.00086 (Pa*s).
The water Prandtl is 5.93690.


#### Steam Properties

In [7]:
# Steam properties
exhaust_steam_properties = _Region2(Tvap,1e-6*Pvap)
exhaust_steam_properties_liq = _Region1(Tvap,1e-6*Pvap)
exhaust_steam_density = 1/exhaust_steam_properties['v']
exhaust_steam_viscosity = _Viscosity(exhaust_steam_density, Tvap)
exhaust_vaporization_heat = 1e3*exhaust_steam_properties['h'] - 1e3*exhaust_steam_properties_liq['h']

print("The exhaust steam density is equal to {number:.{digits}f} kg/m3."\
      .format(number=exhaust_steam_density, digits=5))
print("The exhaust steam vaporization heat is equal to {number:.{digits}f} J/kg."\
      .format(number=exhaust_vaporization_heat, digits=0))

The exhaust steam density is equal to 0.04550 kg/m3.
The exhaust steam vaporization heat is equal to 2411603 J/kg.


#### Hydraulic Calculations

In [8]:
def calculate_darcy(fD0, ep, D, Re):
    """Calculate the darcy friction factor"""
    def idarcy(ifD, ep, D, Re):
        return ifD + 2. * np.log10(ep / 3.72 / D + (2.51 / Re) * ifD )
    sol = root(idarcy, fD0 ** -0.5, args=(ep, D, Re),)
    return  sol.x[0] ** -2

def calculate_fanning(ep, D, Re):
    """Calculate the fanning friction factor"""
    A = (2.457*np.log(((7/Re)**0.9+0.27*ep/D)**-1))**16
    B = (37530/Re)**16
    ff = 2 *((8/Re)**12 + (A+B)**-1.5) ** (1/12)
    return  ff

In [9]:
v = (m /water_density)/(n_tubes * 0.25 * pi * Df**2) # m/s
Re_tube = Df * water_density * v / water_viscosity
ff = calculate_fanning(ep, Df, Re_tube)
g = 9.81
fD_tube = 4 * ff
tau = (1/8)*fD_tube*water_density*v**2
hL = 0.5 * fD_tube * v ** 2 /  (Df * g )
DeltaP = g * water_density * hL
pressure_loss_in_tube = DeltaP * L
Pout = Pin - pressure_loss_in_tube

print("Water mean velocity is {number:.{digits}f} m/s.".format(number=v, digits=2))
print("Pressure Drop is {number:.{digits}f} Pa.".format(number=pressure_loss_in_tube, digits=2))
print("Inlet Pressure is {number:.{digits}f} Pa.".format(number=Pin, digits=2))
print("Outlet Pressure : ", Pout)

Water mean velocity is 2.01 m/s.
Pressure Drop is 24456.76 Pa.
Inlet Pressure is 200000.00 Pa.
Outlet Pressure :  175543.2415056838


#### Temperature Initial Estimates

In [15]:
# The properties are calculated at the mean temperature between bulk and internal surface
Tf = (Tin + Tout) / 2
Ti = Tf
To = (Tin + Tout + Tvap) / 3

#### Thermal calculations - WALL

In [11]:
# Wall conduction
Reswall = np.log(Do / Di) / (2 * pi * kwall * L)

print("The calculated wall resistance x length is {number:.{digits}f} K/W".format(number=Reswall,digits=6))

The calculated wall resistance x length is 0.000087 K/W


#### Thermal calculations - INT

#### LMTD

In [12]:
def calculate_heat(water_flowrate_per_tube,
                   water_heat_capacity,
                   water_inlet_temperature,
                   water_outlet_temperature):
    """Calculate the duty"""
    return water_flowrate_per_tube * water_heat_capacity * (water_outlet_temperature - water_inlet_temperature)


def calculate_LMTD(T1a,T1b,T2a,T2b):
    """Calculate LMTD"""
    dT1 = T1a - T1b
    dT2 = T2a - T2b
    return (dT1 - dT2) / np.log(dT1/dT2)

def calculate_heat_via_LMTD( UA, LMTD):
    """Calculate heat exchanger via LMTD"""
    return UA * LMTD
    
def calculate_heat_balance(x0, exhaust_steam_temperature, water_inlet_temperature, water_flowrate_per_tube, water_heat_capacity, UA ):
    """ Heat balance """
    def heat_balance(x, exhaust_steam_temperature, water_inlet_temperature, water_flowrate_per_tube, water_heat_capacity, UA ):
        water_outlet_temperature = x
        LMTD = calculate_LMTD(exhaust_steam_temperature,water_inlet_temperature,exhaust_steam_temperature,water_outlet_temperature)
        Q1 = calculate_heat_via_LMTD( UA, LMTD)
        Q2 = calculate_heat( water_flowrate_per_tube, water_heat_capacity, water_inlet_temperature, water_outlet_temperature)
        return Q1 - Q2
    sol = root(heat_balance, x0, args=(exhaust_steam_temperature, water_inlet_temperature, water_flowrate_per_tube, water_heat_capacity, UA ))
    return  sol.x[0]

In [13]:
Ahot = pi * Do * L # m2 - Tube heat exchange area
Acold = pi * Df * L # m2 - Tube heat exchange area
Ai = 0.25 * pi * Df**2 # m2 - Free flow area
print("The tube for cold side is {number:.{digits}f} m2".format(number=Acold,digits=2))
print("The tube for hot side is {number:.{digits}f} m2".format(number=Ahot,digits=2))

for k in range(0,10):
    results = []
    for i in range(0,Rows):

        #### EXTERNAL RESISTANCE

        # Calculating properties
        film_properties = _Region1(To[i],1e-6*Pvap)
        film_density = 1/film_properties['v']
        film_entalphy = 1e3*film_properties['h']
        film_heat_capacity = 1e3*film_properties['cp']
        film_conductivity = _ThCond(film_density, To[i])
        film_viscosity = _Viscosity(film_density, To[i])
        film_prandtl = film_heat_capacity * film_viscosity / film_conductivity

        num = (g * film_density * (film_density - exhaust_steam_density) * film_conductivity ** 3. * exhaust_vaporization_heat)
        den = film_viscosity * (Tvap - To[i]) * Do
        hext_row1 = 0.729 * (num / den) ** 0.25
        Resext_row1 = 1 / (pi * Do * hext_row1 * L)

        #fNtub = tubes_per_width**(-1/6)
        fNtub = ((N_wetters[i]+1) ** (5/6) - N_wetters[i] ** (5/6))

        hext = hext_row1 * fNtub
        Resext = Resext_row1 / fNtub


        #### INTERNAL RESISTANCE

        # Calculating properties
        int_film_properties = _Region1(Ti[i],1e-6 * 0.5 * (Pin + Pout))
        int_film_density = 1/int_film_properties['v']
        int_film_entalphy = 1e3*int_film_properties['h']
        int_film_heat_capacity = 1e3*int_film_properties['cp']
        int_film_conductivity = _ThCond(int_film_density, Tf[i])
        int_film_viscosity = _Viscosity(int_film_density, Tf[i])
        int_film_prandtl = int_film_heat_capacity * int_film_viscosity / int_film_conductivity

        # Calculating Darcy
        int_film_Re = Df * int_film_density * v / int_film_viscosity
        int_film_fD = calculate_darcy(fD_tube, ep, Df, int_film_Re)

        # Calculates the Nussel dimensionless number using Petukhov correlation modified by Gnielinski. See Incropera 4th Edition [8.63]
        nusselt = (int_film_fD / 8.) * (int_film_Re - 1000.) * int_film_prandtl / ( 1. + 12.7 * (int_film_fD / 8.) ** 0.5 * (int_film_prandtl ** (2 / 3) - 1.))
        #nusselt = 0.023 * Re ** (4/5) * water_prandtl ** 0.4

        hint = nusselt * water_conductivity / Df
        Resint = 1 / (pi * Df * hint * L)

        # Calculate Overall he
        Restotal = Resext + Resint + Reswall + ResF

        UA = 1/Restotal
        Ucold = UA / Acold
        Uhot = UA / Ahot

        Tout[i] = calculate_heat_balance(Tout[i], Tvap, Tin, m/n_tubes, water_heat_capacity, UA )

        # Calculate LMTD
        LMTD = calculate_LMTD(Tvap,Tin,Tvap,Tout[i])

        Q = calculate_heat_via_LMTD( UA, LMTD)

        Q_at_inlet = (Tvap - Tin) / Restotal
        To_in = Tvap - Q_at_inlet*Resext
        Ti_in = To_in - Q_at_inlet*Reswall
        Tf_in = Ti_in - Q_at_inlet*ResF

        Q_at_outlet = (Tvap - Tout[i]) / Restotal
        To_out = Tvap - Q_at_outlet*Resext
        Ti_out = To_out - Q_at_outlet*Reswall
        Tf_out = Ti_out - Q_at_outlet*ResF

        To[i] = 0.5 * (To_in + To_out)
        Ti[i] = 0.5 * (Ti_in + Ti_out)
        Tf[i] = 0.5 * (Tf_in + Tf_out)

        results.append({
            "UA": UA,
            "LMTD": LMTD,
            "Q": Q,
            "hext": hext,
            "Restotal": Restotal,
            "Resint": Resint,
            "Resext": Resext,
            "Reswall": Reswall,
            "Tf_in": Tf_in,
            "Ti_in": Ti_in,
            "To_in": To_in,
            "Tf_out": Tf_out,
            "Ti_out": Ti_out,
            "To_out": To_out,
            "Tf": Tf[i],
            "Ti": Ti[i],
            "To": To[i],
            "Tout": Tout[i],
        })


for i in (0, -1):
    print(" -- ROW {} --".format(i))
    
    results_i = results[i]
    
    print("The row external convection coefficient is {number:.{digits}f} W/(K*m2)".format(number=results_i["hext"],digits=2))
    print("The last row calculated internal resistance x length is {number:.{digits}f} K/W".format(number=results_i["Resint"],digits=6))
    print("The last row calculated external resistance x length is {number:.{digits}f} K/W".format(number=results_i["Resext"],digits=6))
    print("LMTD is {number:.{digits}f} K".format(number=results_i["LMTD"],digits=2))
    print("The heat rate per tube is {number:.{digits}f} W".format(number=results_i["Q"],digits=2))
    print("U*A per tube is {number:.{digits}f} W/K".format(number=results_i["UA"],digits=2))
    print("The cooling water inlet temperature is {number:.{digits}f} K".format(number=Tin,digits=2))
    print("Cooling water outlet temperature is {number:.{digits}f} W/K".format(number=results_i["Tout"],digits=2))
    print("Mean internal film temperature is {number:.{digits}f} K".format(number=results_i["Ti"],digits=2))
    print("Mean external film temperature is {number:.{digits}f} K".format(number=results_i["To"],digits=2))

The tube for cold side is 0.69 m2
The tube for hot side is 0.75 m2
 -- ROW 0 --
The row external convection coefficient is 13352.12 W/(K*m2)
The last row calculated internal resistance x length is 0.000136 K/W
The last row calculated external resistance x length is 0.000100 K/W
LMTD is 10.39 K
The heat rate per tube is 27239.44 W
U*A per tube is 2621.95 W/K
The cooling water inlet temperature is 295.55 K
Cooling water outlet temperature is 304.12 W/K
Mean internal film temperature is 303.75 K
Mean external film temperature is 307.93 K
 -- ROW -1 --
The row external convection coefficient is 6963.77 W/(K*m2)
The last row calculated internal resistance x length is 0.000138 K/W
The last row calculated external resistance x length is 0.000192 K/W
LMTD is 11.15 K
The heat rate per tube is 23498.08 W
U*A per tube is 2106.56 W/K
The cooling water inlet temperature is 295.55 K
Cooling water outlet temperature is 302.94 W/K
Mean internal film temperature is 302.60 K
Mean external film temperatu

In [14]:
# Calculating final results
A_total = 0
Q_total = 0

for i in range(0, Rows):
    A_total += Acold*tubes_per_width
    Q_total += results[i]["Q"]*tubes_per_width

kvap_calc =  Q_total/exhaust_vaporization_heat

Tout_mean = Tin + Q_total / (m * water_heat_capacity)

kvap = Q_total/exhaust_vaporization_heat # kg/s

print("The total transfer area (cold side) is {number:.{digits}f} m2".format(number=A_total, digits=2))
print("The total steam flowrate is {number:.{digits}f} ton/h".format(number=kvap_calc*3.6, digits=2))
print("The total heat load of condensation is {number:.{digits}f} MW".format(number=1e-6*Q_total, digits=2))
print("Condensate mass flowrate is {number:.{digits}f} kg/s.".format(number=kvap, digits=2))
print("The cooling water CALCULATED mean outlet temperature is {number:.{digits}f} K".format(number=Tout_mean,digits=2))
print("The cooling water EXPECTED mean outlet temperature is {number:.{digits}f} K".format(number=Tout_mean_expected,digits=2))
print("Cooling Water outlet temp deviation is {number:.{digits}f} %".format(number=(Tout_mean-Tout_mean_expected)/(Tout_mean_expected-273.15)*100,digits=2))
print("Steam temp deviation is {number:.{digits}f} %".format(number=(Tvap-Tvap_expected)/(Tvap_expected - 273.15)*100,digits=2))

The total transfer area (cold side) is 2209.77 m2
The total steam flowrate is 119.17 ton/h
The total heat load of condensation is 79.83 MW
Condensate mass flowrate is 33.10 kg/s.
The cooling water CALCULATED mean outlet temperature is 303.35 K
The cooling water EXPECTED mean outlet temperature is 303.35 K
Cooling Water outlet temp deviation is -0.00 %
Steam temp deviation is -0.80 %
